In [1]:
import imgaug.augmenters as iaa
import cv2, os
import glob
import shutil
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [2]:
#JUPYTER VERSION
#comb_path = "../Data/Combined_Images"

#GOOGLE COLAB VERSION
from google.colab import drive
drive.mount('/content/drive')

# Unzipping the dataset file facial-age.zip

#combined_images_path = "/content/drive/MyDrive/Data/Combined_Images.zip"
combined_images_path = "/content/drive/MyDrive/Data/Combined_Images_Augmented.zip"

with ZipFile(combined_images_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images_Augmented")
    print('Done unzipping Combined_Images_Augmented.zip') 

Mounted at /content/drive
Done unzipping Combined_Images_Augmented.zip


In [9]:
comb_path = '../content/Combined_Images_Augmented' 
batch_size = 128

train_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="training", #If should be return the training set (80%) or the validation set (20%)
  seed=41, #Seed should guarantee that train_ds and val_ds doesn't have common images
  shuffle=True,
  image_size=(50, 50),
  batch_size=batch_size,
  color_mode='grayscale'
  )

val_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="validation",
  seed=41, 
  shuffle=True,
  image_size=(50, 50),
  batch_size=batch_size,
  color_mode='grayscale'
  )

test_dataset = val_ds.take(360)
val_ds = val_ds.skip(360)

print('Batches for training -->', train_ds.cardinality())
print('Batches for validating -->', val_ds.cardinality())
print('Batches for testing -->', test_dataset.cardinality()) 

Found 237188 files belonging to 8 classes.
Using 189751 files for training.
Found 237188 files belonging to 8 classes.
Using 47437 files for validation.
Batches for training --> tf.Tensor(1483, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(11, shape=(), dtype=int64)
Batches for testing --> tf.Tensor(360, shape=(), dtype=int64)


In [4]:
#from matplotlib import pyplot as plt
#from matplotlib.pyplot import figure

##figure(figsize=(18, 16))
#gen = imgs[:6]
#for i in range(1, 6):
    #plt.subplot(1,5,i)
    #plt.axis("off")
    #plt.imshow(gen[i])
    #plt.plot()
#plt.show()

In [11]:
model4 = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(50, 50, 1)),
  #data_augmentation,
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'), #kernel_regularizer=regularizers.l2(l=0.01)
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'), #
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'), #
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(), #See reference above
  #tf.keras.layers.GlobalMaxPool2D(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8, activation='softmax')
])

#model.compile(optimizer=tf.keras.optimizers.Adam(),
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              #metrics=tf.keras.metrics.Accuracy())

#Additionaly, if you do not one-hot encode your data, set sparse_categorical_crossentropy as loss and sparse_categorical_accuracy as metric.
model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

# Defining the early stop to monitor the validation loss to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=30
history = model4.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early_stop],
  epochs=epochs,
  shuffle=True
)

Epoch 1/30
1483/1483 [==============================] - 538s 362ms/step - loss: 1.9146 - sparse_categorical_accuracy: 0.2464 - val_loss: 1.8722 - val_sparse_categorical_accuracy: 0.2579
Epoch 2/30
1483/1483 [==============================] - 530s 357ms/step - loss: 1.7978 - sparse_categorical_accuracy: 0.3038 - val_loss: 1.7202 - val_sparse_categorical_accuracy: 0.3478
Epoch 3/30
1483/1483 [==============================] - 521s 351ms/step - loss: 1.7402 - sparse_categorical_accuracy: 0.3259 - val_loss: 1.6995 - val_sparse_categorical_accuracy: 0.3434
Epoch 4/30
1483/1483 [==============================] - 527s 355ms/step - loss: 1.7102 - sparse_categorical_accuracy: 0.3379 - val_loss: 1.6577 - val_sparse_categorical_accuracy: 0.3508
Epoch 5/30
1483/1483 [==============================] - 528s 356ms/step - loss: 1.6886 - sparse_categorical_accuracy: 0.3437 - val_loss: 1.6415 - val_sparse_categorical_accuracy: 0.3604
Epoch 6/30
1483/1483 [==============================] - 521s 351ms/ste

In [6]:
print("MODEL 2 --- With GlobalMax")
model4.evaluate(test_dataset)

MODEL 2 --- With GlobalMax
360/360 [==============================] - 8s 20ms/step - loss: 1.5790 - sparse_categorical_accuracy: 0.3852


[1.578961730003357, 0.3851562440395355]

In [8]:
print("MODEL 2 --- With Flatten")
model4.evaluate(test_dataset)

MODEL 2 --- With Flatten
360/360 [==============================] - 8s 22ms/step - loss: 1.3922 - sparse_categorical_accuracy: 0.4587


[1.3921560049057007, 0.4587022662162781]

In [12]:
print("MODEL 2 --- With Grayscale")
model4.evaluate(test_dataset)

MODEL 2 --- With Grayscale
360/360 [==============================] - 7s 18ms/step - loss: 1.3923 - sparse_categorical_accuracy: 0.4550


[1.3923063278198242, 0.4550130069255829]